In [22]:
import time
import asyncio
import json
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout



In [26]:
SESSION_FILE = "goodreads_session.json"


async def save_session():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)

        context = await browser.new_context(
            viewport={"width": 1280, "height": 800}
        )
        page = await context.new_page()

        # Go to Goodreads login page
        await page.goto("https://www.goodreads.com/user/sign_in")

        print("\n🚨 MANUAL LOGIN REQUIRED 🚨")
        print("1) Log in to Goodreads in the browser")
        print("2) Make sure you are fully logged in")
        print("3) You may see a CAPTCHA — solve it")
        input("👉 Press Enter AFTER login is complete...")

        # Save cookies + local storage
        await context.storage_state(path=SESSION_FILE)
        print(f"✅ Session saved to {SESSION_FILE}")

        await browser.close()

In [28]:
await save_session()


🚨 MANUAL LOGIN REQUIRED 🚨
1) Log in to Goodreads in the browser
2) Make sure you are fully logged in
3) You may see a CAPTCHA — solve it


👉 Press Enter AFTER login is complete... 


✅ Session saved to goodreads_session.json


In [29]:
import json
import os
import random
from playwright.async_api import async_playwright
from dateutil import parser as dateparser

START_URL = "https://www.goodreads.com/book/show/32758901-all-systems-red/reviews"
CHECKPOINT_FILE = "goodreads_reviews_checkpoint.jsonl"

# --------------------------------
# Utilities
# --------------------------------

def normalize_date(date_str):
    if not date_str:
        return None
    try:
        return dateparser.parse(date_str).date().isoformat()
    except:
        return date_str.strip()


def review_signature(r):
    return (
        (r.get("username") or "").lower().strip(),
        (r.get("date") or "").strip(),
        r.get("text", "")[:200].lower()
    )


def load_existing_signatures():
    seen = set()
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
            for line in f:
                r = json.loads(line)
                seen.add(review_signature(r))
    return seen


def append_checkpoint(reviews):
    with open(CHECKPOINT_FILE, "a", encoding="utf-8") as f:
        for r in reviews:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")


# --------------------------------
# Page interaction helpers
# --------------------------------

async def expand_review_text(page):
    expanders = await page.query_selector_all(
        "article a:has-text('Show more'), article button:has-text('Show more')"
    )
    for el in expanders:
        try:
            await page.evaluate("(e) => e.click()", el)
            await page.wait_for_timeout(120)
        except:
            pass


async def extract_current_reviews(page):
    return await page.eval_on_selector_all(
        "article, div[data-testid='review']",
        r"""
        cards => cards.map(card => {
            const username =
                card.querySelector('a[href*="/user/show"]')?.innerText?.trim() ?? null;

            let date = null;
            const spans = Array.from(card.querySelectorAll('span'))
                .map(s => s.innerText.trim())
                .filter(t =>
                    t.match(/\d{4}|ago|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec/)
                );

            if (spans.length > 0) {
                date = spans[spans.length - 1];
            }

            return {
                username,
                date,
                text: card.innerText.trim()
            };
        })
        """
    )


async def find_page_level_load_more(page):
    buttons = await page.query_selector_all("button:has-text('Show more')")
    for btn in buttons:
        inside_review = await btn.evaluate(
            "el => el.closest('article, div[data-testid=\"review\"]') !== null"
        )
        if not inside_review:
            return btn
    return None


# --------------------------------
# Main scraper
# --------------------------------

async def scrape_goodreads_reviews(
    max_pages=5000,
    manual_unlock=True
):
    seen_reviews = load_existing_signatures()
    print(f"Resuming with {len(seen_reviews)} reviews already saved")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)

        # 🔑 SHORT viewport is REQUIRED
        context = await browser.new_context(
            viewport={"width": 1280, "height": 700}
        )
        page = await context.new_page()

        await page.goto(START_URL, wait_until="domcontentloaded")
        await page.wait_for_timeout(4000)

        # --------------------------------
        # Manual unlock (ONCE)
        # --------------------------------
        if manual_unlock:
            print("\n🚨 MANUAL STEP (ONCE PER SESSION) 🚨")
            print("• Scroll normally")
            print("• Click 'Show more reviews' ONCE")
            print("• Confirm reviews change")
            input("👉 Press Enter to continue automation...")

        # --------------------------------
        # Pagination loop (replace model)
        # --------------------------------
        for page_num in range(max_pages):
            print(f"\n--- Review batch {page_num + 1} ---")

            # Expand review-level text first
            await expand_review_text(page)

            current_reviews = await extract_current_reviews(page)
            if not current_reviews:
                print("No reviews found. Stopping.")
                break

            new_reviews = []
            for r in current_reviews:
                r["date"] = normalize_date(r["date"])
                sig = review_signature(r)
                if sig not in seen_reviews:
                    seen_reviews.add(sig)
                    new_reviews.append(r)

            if new_reviews:
                append_checkpoint(new_reviews)
                print(
                    f"Saved {len(new_reviews)} new reviews "
                    f"(total saved: {len(seen_reviews)})"
                )
            else:
                print("No new reviews on this page")

            load_more = await find_page_level_load_more(page)
            if not load_more:
                print("No 'Show more reviews' button found. Done.")
                break

            before_text = current_reviews[0]["text"]

            await load_more.scroll_into_view_if_needed()
            await page.wait_for_timeout(500)
            await page.evaluate("(b) => b.click()", load_more)

            # Polite delay (human-like)
            await page.wait_for_timeout(random.randint(1200, 2200))

            after_reviews = await extract_current_reviews(page)
            if after_reviews and after_reviews[0]["text"] == before_text:
                print("Page content did not change. Stopping.")
                break

        await browser.close()


# --------------------------------
# Run
# --------------------------------

# Start small first (e.g., max_pages=5), then increase to ~5000
# reviews ≈ 10 per page after first page
#
# await scrape_goodreads_reviews(max_pages=5000, manual_unlock=True)




In [32]:
! rm ./goodreads_reviews_checkpoint.jsonl

In [33]:
await scrape_goodreads_reviews(max_pages=5000, manual_unlock=False) # manual_unlock=True GPT says should be False when logged in

Resuming with 0 reviews already saved

--- Review batch 1 ---
Saved 31 new reviews (total saved: 31)

--- Review batch 2 ---
Saved 30 new reviews (total saved: 61)

--- Review batch 3 ---
Saved 30 new reviews (total saved: 91)

--- Review batch 4 ---
Saved 30 new reviews (total saved: 121)

--- Review batch 5 ---
Saved 30 new reviews (total saved: 151)

--- Review batch 6 ---
Saved 30 new reviews (total saved: 181)

--- Review batch 7 ---
Saved 30 new reviews (total saved: 211)

--- Review batch 8 ---
Saved 30 new reviews (total saved: 241)

--- Review batch 9 ---
Saved 30 new reviews (total saved: 271)

--- Review batch 10 ---
Saved 30 new reviews (total saved: 301)

--- Review batch 11 ---
Saved 30 new reviews (total saved: 331)

--- Review batch 12 ---
Saved 30 new reviews (total saved: 361)

--- Review batch 13 ---
Saved 30 new reviews (total saved: 391)

--- Review batch 14 ---
Saved 30 new reviews (total saved: 421)

--- Review batch 15 ---
Saved 30 new reviews (total saved: 451)

In [19]:
path = "goodreads_reviews_checkpoint.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        review = json.loads(line)
        print(review)
        if i == 4:
            break

{'username': '', 'date': '2017-08-29', 'text': 'K.J. Charles\nAuthor\xa0\n65 books\n12.1k followers\nFollow\nRead\nAugust 29, 2017\nMagnificent. Tor.com continue to rock it with this fantastic tale of a self-hacked cyborg security unit with a gloriously bad attitude. It just wants to sit there downloading soaps but HEY HO SABOTAGE, PEOPLE TO SAVE, PEOPLE TO KILL.\n\nTerrific writing, lovely characterisation, very funny, and there will be more! Consider me glued.\nfun-fun-fun\n \nnovella\n \noh-god-where-is-the-next-book\n \n...more\n128 likes\n3 comments\nLike\nComment'}
{'username': '', 'date': 'read-in-2019', 'text': "Melanie (meltotheany)\n1,195 reviews\n102k followers\nFollow\nAugust 23, 2019\nI really enjoyed this! And it had such a great discussion on what it means to be human and what we are all willing to do in the name of trust. I can't wait to continue on with this series! <3\n\nBlog | Instagram | Twitter | Tumblr | Youtube | Twitch\n\nBuddy read with Lea! ❤\n\n❤ I read this 

In [11]:
with open("dates_151_goodreads_reviews.json", "w", encoding="utf-8") as f:
    json.dump(reviews, f, ensure_ascii=False, indent=2)

In [25]:
len(reviews)

31

In [13]:
from collections import Counter


In [14]:
texts = [r["text"].strip() for r in reviews if r.get("text")]
text_counts = Counter(texts)

exact_dupes = {t: c for t, c in text_counts.items() if c > 1}

print(f"Exact text duplicates: {len(exact_dupes)}")

Exact text duplicates: 0


In [34]:
import os, json, random
from playwright.async_api import async_playwright
from dateutil import parser as dateparser

BASE_REVIEWS_URL = "https://www.goodreads.com/book/show/32758901-all-systems-red/reviews"
SESSION_FILE = "goodreads_session.json"  # created by context.storage_state(...)

def normalize_date(date_str):
    if not date_str:
        return None
    try:
        return dateparser.parse(date_str).date().isoformat()
    except:
        return date_str.strip()

def review_signature(r):
    return (
        (r.get("username") or "").lower().strip(),
        (r.get("date") or "").strip(),
        (r.get("text") or "")[:200].lower(),
    )

def load_existing_signatures(checkpoint_file):
    seen = set()
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r", encoding="utf-8") as f:
            for line in f:
                r = json.loads(line)
                seen.add(review_signature(r))
    return seen

def append_checkpoint(checkpoint_file, reviews):
    with open(checkpoint_file, "a", encoding="utf-8") as f:
        for r in reviews:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

async def expand_review_text(page):
    expanders = await page.query_selector_all(
        "article a:has-text('Show more'), article button:has-text('Show more')"
    )
    for el in expanders:
        try:
            await page.evaluate("(e) => e.click()", el)
            await page.wait_for_timeout(120)
        except:
            pass

async def extract_current_reviews(page):
    # Date extraction from metadata-ish spans (works better than <time> on this UI)
    return await page.eval_on_selector_all(
        "article, div[data-testid='review']",
        r"""
        cards => cards.map(card => {
            const username =
                card.querySelector('a[href*="/user/show"]')?.innerText?.trim() ?? null;

            let date = null;
            const spans = Array.from(card.querySelectorAll('span'))
                .map(s => s.innerText.trim())
                .filter(t =>
                    t.match(/\d{4}|ago|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec/)
                );

            if (spans.length > 0) date = spans[spans.length - 1];

            return { username, date, text: card.innerText.trim() };
        })
        """
    )

async def find_page_level_load_more(page):
    # Page-level pagination control (exclude per-review expanders)
    buttons = await page.query_selector_all("button:has-text('Show more')")
    for btn in buttons:
        inside_review = await btn.evaluate(
            "el => el.closest('article, div[data-testid=\"review\"]') !== null"
        )
        if not inside_review:
            return btn
    return None

async def scrape_after_manual_sort(checkpoint_file, max_batches=5000):
    """
    Opens Goodreads reviews page, waits for you to manually choose sort order,
    then paginates and saves to checkpoint_file (JSONL).
    """
    seen = load_existing_signatures(checkpoint_file)
    print(f"Resuming {checkpoint_file} with {len(seen)} reviews already saved")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)

        # Short viewport so controls are reachable
        context = await browser.new_context(
            viewport={"width": 1280, "height": 700},
            storage_state=SESSION_FILE,  # logged-in session
        )
        page = await context.new_page()

        await page.goto(BASE_REVIEWS_URL, wait_until="domcontentloaded")
        await page.wait_for_timeout(3000)

        # MANUAL SORT STEP
        print("\n🚨 MANUAL STEP 🚨")
        print("1) In the browser, set sort to the desired mode (Newest OR Oldest).")
        print("2) Wait until the visible reviews reorder.")
        input("👉 Press Enter to start automated scraping...")

        # Optional: capture the post-sort URL (often same URL, but useful if it changes)
        print("Current URL after your sort:", page.url)

        for batch in range(max_batches):
            # Expand review text before extraction
            await expand_review_text(page)

            current = await extract_current_reviews(page)
            if not current:
                print("No reviews found; stopping.")
                break

            # Collect & checkpoint new reviews
            new_reviews = []
            for r in current:
                r["date"] = normalize_date(r.get("date"))
                sig = review_signature(r)
                if sig not in seen:
                    seen.add(sig)
                    new_reviews.append(r)

            if new_reviews:
                append_checkpoint(checkpoint_file, new_reviews)
                print(f"Batch {batch+1}: saved {len(new_reviews)} (total saved: {len(seen)})")
            else:
                print(f"Batch {batch+1}: no new reviews on this batch")

            # Click "Show more reviews" (page-level)
            load_more = await find_page_level_load_more(page)
            if not load_more:
                print("No page-level 'Show more reviews' button found; done.")
                break

            before_head = current[0]["text"][:200]

            await load_more.scroll_into_view_if_needed()
            await page.wait_for_timeout(400)
            await page.evaluate("(b) => b.click()", load_more)

            # Polite delay
            await page.wait_for_timeout(random.randint(1200, 2200))

            after = await extract_current_reviews(page)
            if after and after[0]["text"][:200] == before_head:
                print("Page content did not change (hit window cap / end). Stopping.")
                break

        await browser.close()

# ---- Run newest then oldest (two manual runs) ----
# 1) Run this, set sort to NEWEST manually, then press Enter
# await scrape_after_manual_sort("reviews_newest.jsonl", max_batches=5000)

# 2) Run again, set sort to OLDEST manually, then press Enter
# await scrape_after_manual_sort("reviews_oldest.jsonl", max_batches=5000)


In [55]:
# choose newest, click show more, click enter.
await scrape_after_manual_sort("reviews_newest.jsonl", max_batches=5000)

Resuming reviews_newest.jsonl with 0 reviews already saved

🚨 MANUAL STEP 🚨
1) In the browser, set sort to the desired mode (Newest OR Oldest).
2) Wait until the visible reviews reorder.


👉 Press Enter to start automated scraping... 


Current URL after your sort: https://www.goodreads.com/book/show/32758901-all-systems-red/reviews
Batch 1: saved 31 (total saved: 31)
Batch 2: saved 30 (total saved: 61)
Batch 3: saved 30 (total saved: 91)
Batch 4: saved 30 (total saved: 121)
Batch 5: saved 30 (total saved: 151)
Batch 6: saved 30 (total saved: 181)
Batch 7: saved 30 (total saved: 211)
Batch 8: saved 30 (total saved: 241)
Batch 9: saved 30 (total saved: 271)
Batch 10: saved 30 (total saved: 301)
Batch 11: saved 30 (total saved: 331)
Batch 12: saved 30 (total saved: 361)
Batch 13: saved 30 (total saved: 391)
Batch 14: saved 30 (total saved: 421)
Batch 15: saved 30 (total saved: 451)
Batch 16: saved 30 (total saved: 481)
Batch 17: saved 30 (total saved: 511)
Batch 18: saved 30 (total saved: 541)
Page content did not change (hit window cap / end). Stopping.


In [39]:
path = "reviews_newest.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        review = json.loads(line)
        print(review)
        if i == 4:
            break

{'username': '', 'date': '2025-12-21', 'text': 'katy ♡\n186 reviews\nFollow\nDecember 21, 2025\ni loved this. murderbot is just so cutie and funny. i just think bc it’s soo short that i ended up confused a bit. i needed this to be longer with more explanations. i think that’s just a skill issue tho. i know there’s 7 more books, so i will absolutely be reading those at some point\n3 likes\nLike\nComment'}
{'username': '', 'date': '2025-12-21', 'text': 'namey\n2 reviews\nFollow\nDecember 21, 2025\nfelt like a draft, boring plot, somewhat fun at times\nyou get no description of what anything or anyone looks like or is like.\nthe humor isnt anything special.\nif it was longer it might have been better.\ni plan to read the next volume anyway.\nim bad at reviews\nLike\nComment'}
{'username': '', 'date': '2025-12-21', 'text': "Keila\n182 reviews\n4 followers\nFollow\nDecember 21, 2025\nMy only peeve is that this book isn't any longer, but it put me off my miserable reading slump so I guess it

In [45]:
path = "./seems-good/reviews_newest_1000ish.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        review = json.loads(line)
        print(review)
        if i == 4:
            break

{'username': '', 'date': '2025-12-21', 'text': 'katy ♡\n186 reviews\nFollow\nDecember 21, 2025\ni loved this. murderbot is just so cutie and funny. i just think bc it’s soo short that i ended up confused a bit. i needed this to be longer with more explanations. i think that’s just a skill issue tho. i know there’s 7 more books, so i will absolutely be reading those at some point\n3 likes\nLike\nComment'}
{'username': '', 'date': '2025-12-21', 'text': 'namey\n2 reviews\nFollow\nDecember 21, 2025\nfelt like a draft, boring plot, somewhat fun at times\nyou get no description of what anything or anyone looks like or is like.\nthe humor isnt anything special.\nif it was longer it might have been better.\ni plan to read the next volume anyway.\nim bad at reviews\nLike\nComment'}
{'username': '', 'date': '2025-12-21', 'text': "Keila\n182 reviews\n4 followers\nFollow\nDecember 21, 2025\nMy only peeve is that this book isn't any longer, but it put me off my miserable reading slump so I guess it

In [49]:
# choose oldest, click show more, click enter.
await scrape_after_manual_sort("reviews_oldest.jsonl", max_batches=5000)

Resuming reviews_oldest.jsonl with 0 reviews already saved

🚨 MANUAL STEP 🚨
1) In the browser, set sort to the desired mode (Newest OR Oldest).
2) Wait until the visible reviews reorder.


👉 Press Enter to start automated scraping... 


Current URL after your sort: https://www.goodreads.com/book/show/32758901-all-systems-red/reviews
Batch 1: saved 31 (total saved: 31)
Batch 2: saved 30 (total saved: 61)
Batch 3: saved 30 (total saved: 91)
Batch 4: saved 30 (total saved: 121)
Batch 5: saved 30 (total saved: 151)
Batch 6: saved 30 (total saved: 181)
Batch 7: saved 30 (total saved: 211)
Batch 8: saved 30 (total saved: 241)
Batch 9: saved 30 (total saved: 271)
Batch 10: saved 30 (total saved: 301)
Batch 11: saved 30 (total saved: 331)
Batch 12: saved 30 (total saved: 361)
Batch 13: saved 30 (total saved: 391)
Batch 14: saved 30 (total saved: 421)
Batch 15: saved 30 (total saved: 451)
Batch 16: saved 30 (total saved: 481)
Batch 17: saved 30 (total saved: 511)
Batch 18: saved 30 (total saved: 541)
Batch 19: saved 30 (total saved: 571)
Batch 20: saved 30 (total saved: 601)
Batch 21: saved 30 (total saved: 631)
Batch 22: saved 30 (total saved: 661)
Batch 23: saved 30 (total saved: 691)
Batch 24: saved 30 (total saved: 721)
Ba

In [50]:
path = "reviews_oldest.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        review = json.loads(line)
        print(review)
        if i == 4:
            break

{'username': '', 'date': '2024-02-22', 'text': 'Jessica\n200 reviews\n6 followers\nFollow\nFebruary 22, 2024\nA great read -- an interesting concept well-executed and an exciting plot, too. I do love a (slightly) unreliable narrator. I really hope this will be a series!\nfavorites\n \nfiction\n \nscience-fiction\n1 like\nLike\nComment'}
{'username': '', 'date': '2017-05-03', 'text': 'Silea\n227 reviews\n14 followers\nFollow\nMay 3, 2017\nFast, fun, snarky. Eagerly awaiting Murderbot Diaries #2.\nfiction\nLike\nComment'}
{'username': '', 'date': '2017-05-03', 'text': "Esteefee\nAuthor\xa0\n3 books\n6 followers\nFollow\nMay 3, 2017\nMurderbot is so incredibly endearing -- I didn't stop grinning the entire time I was reading this novel. The Murderbot's social awkwardness with humans, its reliance on media rather than its own subpar training modules, its grim understanding of its own social standing, and of course the bumbling humans who want so desperately to help it. And all of this embe